In [2]:
from collections import defaultdict, Counter
import itertools
import numpy as np
import json
import random as rd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

## Load an Example Network

In [1]:
# Download a network about dolphin friendship
!wget http://www-personal.umich.edu/~mejn/netdata/dolphins.zip
!unzip dolphins.zip

--2023-07-20 18:44:35--  http://www-personal.umich.edu/~mejn/netdata/dolphins.zip
Resolving www-personal.umich.edu (www-personal.umich.edu)... 141.211.243.103
Connecting to www-personal.umich.edu (www-personal.umich.edu)|141.211.243.103|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1868 (1.8K) [application/zip]
Saving to: ‘dolphins.zip’

dolphins.zip        100%[===================>]   1.82K  --.-KB/s    in 0s      

2023-07-20 18:44:35 (7.01 MB/s) - ‘dolphins.zip’ saved [1868/1868]

Archive:  dolphins.zip
  inflating: dolphins.gml            
  inflating: dolphins.txt            


In [3]:
# read the network from it .gml format
G = nx.read_gml("dolphins.gml")

In [6]:
for i in G.nodes():
  print(i)

Beak
Beescratch
Bumper
CCL
Cross
DN16
DN21
DN63
Double
Feather
Fish
Five
Fork
Gallatin
Grin
Haecksel
Hook
Jet
Jonah
Knit
Kringel
MN105
MN23
MN60
MN83
Mus
Notch
Number1
Oscar
Patchback
PL
Quasi
Ripplefluke
Scabs
Shmuddel
SMN5
SN100
SN4
SN63
SN89
SN9
SN90
SN96
Stripes
Thumper
Topless
TR120
TR77
TR82
TR88
TR99
Trigger
TSN103
TSN83
Upbang
Vau
Wave
Web
Whitetip
Zap
Zig
Zipfel


In [7]:
for i in G.edges():
  print(i)

('Beak', 'Fish')
('Beak', 'Grin')
('Beak', 'Haecksel')
('Beak', 'SN9')
('Beak', 'SN96')
('Beak', 'TR77')
('Beescratch', 'Jet')
('Beescratch', 'Knit')
('Beescratch', 'Notch')
('Beescratch', 'Number1')
('Beescratch', 'Oscar')
('Beescratch', 'SN100')
('Beescratch', 'SN90')
('Beescratch', 'Upbang')
('Bumper', 'Fish')
('Bumper', 'SN96')
('Bumper', 'Thumper')
('Bumper', 'Zipfel')
('CCL', 'Double')
('CCL', 'Grin')
('CCL', 'Zap')
('Cross', 'Trigger')
('DN16', 'Feather')
('DN16', 'Gallatin')
('DN16', 'Wave')
('DN16', 'Web')
('DN21', 'Feather')
('DN21', 'Gallatin')
('DN21', 'Jet')
('DN21', 'Upbang')
('DN21', 'Wave')
('DN21', 'Web')
('DN63', 'Knit')
('DN63', 'Number1')
('DN63', 'PL')
('DN63', 'SN9')
('DN63', 'Upbang')
('Double', 'Kringel')
('Double', 'Oscar')
('Double', 'SN4')
('Double', 'Topless')
('Double', 'Zap')
('Feather', 'Gallatin')
('Feather', 'Jet')
('Feather', 'Ripplefluke')
('Feather', 'SN90')
('Feather', 'Web')
('Fish', 'Patchback')
('Fish', 'SN96')
('Fish', 'TR77')
('Five', 'Trigger'


## Generic Network Layout

The sprinng layout is the most commonly used layout.

In [8]:
# Spring Layout - the most generic layout
pos = nx.spring_layout(G, dim=3)

In [9]:
pos

{'Beak': array([-0.21207921, -0.10804016,  0.00833113]),
 'Beescratch': array([ 0.33885294,  0.05501732, -0.18032804]),
 'Bumper': array([-0.21010153, -0.23463817,  0.24299816]),
 'CCL': array([-0.28302113,  0.1851897 , -0.03483805]),
 'Cross': array([-0.66030895, -0.28637434, -0.13978946]),
 'DN16': array([ 0.8462383 ,  0.07444975, -0.1351763 ]),
 'DN21': array([ 0.735553  ,  0.09651103, -0.18459139]),
 'DN63': array([ 0.21451431,  0.01957558, -0.11611215]),
 'Double': array([-0.18020786,  0.06696409, -0.02840589]),
 'Feather': array([ 0.77000563,  0.01791724, -0.18806821]),
 'Fish': array([-0.24519172, -0.21701292,  0.08274855]),
 'Five': array([-0.69326827, -0.19733826, -0.19390865]),
 'Fork': array([-0.35932305,  0.17300056,  0.44131959]),
 'Gallatin': array([ 0.71773375,  0.01035701, -0.16358275]),
 'Grin': array([-0.32558259,  0.03896462,  0.12600574]),
 'Haecksel': array([-0.34048543, -0.02501621, -0.11536815]),
 'Hook': array([-0.24441633,  0.05447741,  0.19361454]),
 'Jet': ar


## Plotly Preview


### Define The Function `plotly_preview`

In [10]:
def plotly_preview(G, pos, node_colors=None, edge_colors=None):

    """
    Generate a 3D network visualization using Plotly.

    Parameters:
        - G (networkx.Graph): The graph object representing the network.
        - pos3D (dict): A dictionary mapping each node to its 3D coordinates (x, y, z).
        - node_colors (dict, optional): A dictionary mapping nodes to custom hex colors.
                                        If not provided, the default color is '#40b9d4'.
        - edge_colors (dict, optional): A dictionary mapping edges to custom hex colors.
                                        If not provided, the default color is 'gray'.

    Output:
        - An HTML file named 'network_visualization.html' is generated, which opens in a web browser.

    Example usage:
        node_colors = {1: '#ff0000', 2: '#00ff00', 3: '#0000ff'}
        edge_colors = {(1, 2): '#ff00ff', (2, 3): '#ffff00'}
        pos = nx.spring_layout(G,dim=3)
        plotly_review_2(G, pos, node_colors, edge_colors)
    """

    # Create a Plotly figure
    fig = go.Figure()

    # Add nodes to the figure
    for node in G.nodes():
        x, y, z = pos[node]
        color = node_colors[node] if node_colors and node in node_colors else '#40b9d4'
        fig.add_trace(go.Scatter3d(
            x=[x],
            y=[y],
            z=[z],
            mode='markers',
            marker=dict(
                size=5,
                color=color,
            ),
            name=str(node),
            text=str(node),
            hovertemplate=None,
        ))

    # Add edges to the figure
    for edge in G.edges():
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_color = edge_colors[edge] if edge_colors and edge in edge_colors else 'gray'
        fig.add_trace(go.Scatter3d(
            x=[x0, x1],
            y=[y0, y1],
            z=[z0, z1],
            mode='lines',
            line=dict(
                color=edge_color,
                width=1,
            ),
            hoverinfo='none',
        ))

    # Set layout options
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        showlegend=False,
        hovermode='closest',
        margin=dict(l=0, r=0, b=0, t=0),
    )

    # Display the plot inline in the notebook
    iplot(fig)

### Use the Function

In [11]:
plotly_preview(G, pos)


## Export JSON




### Define Function `make_json`

In [ ]:

def make_json(name, network, positions,
              node_color = '#40b9d4',
              link_color = '#999999',
              annotations = 'None',
              communities = 'None'):
    """
    Generates a JSON file from a given network graph using the specified parameters.

    Args:
        name (str, optional): Name of the graph.
        network (networkx.Graph): Network graph object.
        positions (dict): Dictionary mapping node IDs to their positions.
        node_color (dict): Dictionary mapping node IDs to their (hex-)colors.
        link_color (str or dict): (Hex-)color value for all links in the graph or
                                   dict with node tuple as key and hex color as value.
        communities (dict): 'None' for no communities (default) or dictionary mapping
                            node IDs to their corresponding community ID.
        annotations (dict): Dictionary mapping node IDs to a list of annotations.

    Returns:
        None

    """

    # --------------------------
    # Generate VR GRAPH
    # --------------------------
    GVR = nx.Graph()
    GVR.graph['name'] = name

    # --------------------------------------
    # LOOKUP FOR NODE NAMES INTO IDs and vv
    # --------------------------------------
    d_idx_node = {}
    d_node_idx = {}
    for i, node in enumerate(sorted(network.nodes())):
        d_idx_node[i] = node
        d_node_idx[node] = i
    GVR.add_nodes_from(d_idx_node.keys())

    for edge in network.edges()(data=True):
        GVR.add_edge(d_node_idx[edge[0]],d_node_idx[edge[1]])

    # --------------------------
    # POS
    # --------------------------
    if isinstance(positions[next(iter(positions))], list):
        pass
    else:
        for key in positions:
            positions[key] = positions[key].tolist()

    posG = {d_node_idx[node]: list(xyz) for node, xyz in positions.items()}
    nx.set_node_attributes(GVR, posG, name="pos")

    # # --------------------------
    # # CLUSTER
    # # --------------------------
    if communities == 'None':
        dict_for_cluster = dict(zip(d_idx_node.keys(), [0 for _ in d_idx_node.keys()]))
    else:
        d_VRids_cluster = {d_node_idx[node]: str(cl_id) for node, cl_id in communities.items()}
        nx.set_node_attributes(GVR, d_VRids_cluster, name="cluster")


    # --------------------------
    # NODE COLOR
    # --------------------------
    d_node_colors={}

    if isinstance(node_color, dict):
        for nodeid in GVR.nodes():
            d_node_colors[nodeid] = node_color[d_idx_node[nodeid]]
    else:
        for nodeid in GVR.nodes():
            d_node_colors[nodeid] = node_color

    nx.set_node_attributes(GVR, d_node_colors, name="nodecolor")

    # --------------------------
    # LINK COLOR
    # --------------------------
    if isinstance(link_color, dict):
        # for different link colors
        d_edge_color = {}
        for a,b in GVR.edges():
            try:
                color = link_color[(d_idx_node[a],d_idx_node[b])]
            except KeyError:
                color = link_color[(d_idx_node[b],d_idx_node[a])]
            d_edge_color[(a,b)] = color
    else:
        # for unique link colors
        d_edge_color = {}
        for a,b in GVR.edges():
            d_edge_color[(a,b)] = link_color

    nx.set_edge_attributes(GVR, d_edge_color, name="linkcolor")

    # --------------------------
    # NODE ANNOTATION
    # --------------------------
    if isinstance(annotations, dict):

        l_annotations = [[str(d_idx_node[nodeid])] + [ annotation for annotation in annotations[d_idx_node[nodeid]]] for nodeid in sorted(GVR.nodes())]
        d_annotations = dict(zip(sorted(GVR.nodes()), l_annotations))
    else:
        d_annotations = {nodeid: [str(d_idx_node[nodeid])] for nodeid in GVR.nodes()}

    nx.set_node_attributes(GVR, d_annotations, name="annotation")

    # --------------------------
    # MAKE JSON fo uploader
    # --------------------------

    G_json = json.dumps(nx.node_link_data(GVR))

    with open(GVR.name, "w") as outfile:
        outfile.write(G_json)


### Run the Function



In [ ]:
make_json("minimal_example.json", G, pos)

In [ ]:
import google.colab.files
google.colab.files.download("minimal_example.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>